![alt text for screen readers](https://intro-to-btt-using-python-assets.s3.amazonaws.com/bladesight_logo_horizontal_ORIGINAL.jpg).
# Chapter 6: Data Zeroing and Filtering

In [ ]:
# Run this cell if you have not installed the `bladesight` package yet
%pip install bladesight
## NBNBNB! You may need to restart the kernel after installing the package! If you 
# installed it through the Kernel, you can skip this cell.

In [ ]:
# If plotly is not installed
%pip install plotly
## NBNBNB! You may need to restart the kernel after installing the package! If you 
# installed it through the Kernel, you can skip this cell.

In [ ]:
# If Numba is not installed
%pip install numba
## NBNBNB! You may need to restart the kernel after installing the package! If you 
# installed it through the Kernel, you can skip this cell.

In [ ]:
# If scipy is not installed
%pip install scipy
## NBNBNB! You may need to restart the kernel after installing the package! If you 
# installed it through the Kernel, you can skip this cell.

In [ ]:
from bladesight import Datasets
from bladesight.btt import get_rotor_blade_AoAs
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.signal import detrend


In [ ]:
ds = Datasets["data/intro_to_btt/intro_to_btt_ch06"]
df_opr_zero_crossings = ds['table/opr_zero_crossings']
df_prox_1 = ds['table/prox_1_toas']
df_prox_2 = ds['table/prox_2_toas']
df_prox_3 = ds['table/prox_3_toas']


In [ ]:
blade_AoAs = get_rotor_blade_AoAs(
    df_opr_zero_crossings, 
    [
        df_prox_1, 
        df_prox_2,
        df_prox_3
    ],
    probe_spacings=np.deg2rad([10,20]),
    B=5
)

### Copy the first blade's df to use for the remainder of the worksheet

In [ ]:
df = blade_AoAs[0].copy(deep=True)

### Raw AoAs

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

for prox_no in range(1, 4):
    fig.add_trace(go.Scattergl(
        x=df['n'],
        y=df[f'AoA_p{prox_no}'],
        mode='markers+lines',
        name=f'Prox {prox_no}'
    ), secondary_y=False)

fig.add_trace(go.Scattergl(
    x=df['n'],
    y=df['Omega']*60/(2*np.pi),
    mode='markers+lines',
    name='Shaft speed'
), secondary_y=True)
fig.update_layout(
    title='Blade AoA',
    xaxis_title='Shaft revolution',
    yaxis_title='AoA [rad]',
    legend_title='Probe'
)
fig.show()

## Normalization and scaling

In [ ]:
df["AoA_norm_p1"] = df[f"AoA_p1"].mean() - df[f"AoA_p1"] 
df["AoA_norm_p2"] = df[f"AoA_p2"].mean() - df[f"AoA_p2"]
df["AoA_norm_p3"] = df[f"AoA_p3"].mean() - df[f"AoA_p3"]


In [ ]:
R = 164000 
df["x_p1"] = R * df["AoA_norm_p1"]
df["x_p2"] = R * df["AoA_norm_p2"]
df["x_p3"] = R * df["AoA_norm_p3"]

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
for prox_no in range(1, 4):
    fig.add_trace(go.Scattergl(
        x=df['n'],
        y=df[f'x_p{prox_no}'],
        mode='markers+lines',
        name=f'Prox {prox_no}'
    ), secondary_y=False)

fig.add_trace(go.Scattergl(
    x=df['n'],
    y=df['Omega']*60/(2*np.pi),
    mode='markers+lines',
    name='Shaft speed'
), secondary_y=True)

fig.update_layout(
    title='Tip deflections',
    xaxis_title='Shaft revolution',
    yaxis_title='x [μm]',
    legend_title='Probe'
)
fig.show()

In [ ]:
bps = np.array([217, 1128, 1436, 1784, 2670])
df[f"x_p1"] = detrend(
    df["x_p1"],
    bp=bps
)
df[f"x_p2"] = detrend(
    df["x_p2"],
    bp=bps
)
df[f"x_p3"] = detrend(
    df["x_p3"],
    bp=bps
)


In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
for prox_no in range(1, 4):
    fig.add_trace(go.Scattergl(
        x=df['n'],
        y=df[f'x_p{prox_no}'],
        mode='markers+lines',
        name=f'Prox {prox_no}'
    ), secondary_y=False)

fig.add_trace(go.Scattergl(
    x=df['n'],
    y=df['Omega']*60/(2*np.pi),
    mode='markers+lines',
    name='Shaft speed'
), secondary_y=True)

fig.update_layout(
    title='Detrended tip deflections',
    xaxis_title='Shaft revolution',
    yaxis_title='x [μm]',
    legend_title='Probe'
)
fig.show()

### Order domain polynomial detrending

In [ ]:
df["x_p1"] = R * df["AoA_norm_p1"]
df["x_p2"] = R * df["AoA_norm_p2"]
df["x_p3"] = R * df["AoA_norm_p3"]

poly_order = 11 
p1 = np.polyfit(df['Omega'], df['x_p1'], poly_order)
p2 = np.polyfit(df['Omega'], df['x_p2'], poly_order)
p3 = np.polyfit(df['Omega'], df['x_p3'], poly_order)

df["x_p1"] = df["x_p1"] - np.polyval(p1, df['Omega'])
df["x_p2"] = df["x_p2"] - np.polyval(p2, df['Omega'])
df["x_p3"] = df["x_p3"] - np.polyval(p3, df['Omega'])


In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
for prox_no in range(1, 4):
    fig.add_trace(go.Scattergl(
        x=df['n'],
        y=df[f'x_p{prox_no}'],
        mode='markers+lines',
        name=f'Prox {prox_no}'
    ), secondary_y=False)

fig.add_trace(go.Scattergl(
    x=df['n'],
    y=df['Omega']*60/(2*np.pi),
    mode='markers+lines',
    name='Shaft speed'
), secondary_y=True)

fig.update_layout(
    title='Detrended tip deflections',
    xaxis_title='Shaft revolution',
    yaxis_title='x [μm]',
    legend_title='Probe'
)
fig.show()

## Filtering

### Butterworth filter


In [ ]:
from scipy.signal import butter, filtfilt

butterworth_filter = butter(N=2, Wn=0.3) 

df["x_p1_filt_butter"] = filtfilt(*butterworth_filter, df["x_p1"]) 
df["x_p2_filt_butter"] = filtfilt(*butterworth_filter, df["x_p2"])
df["x_p3_filt_butter"] = filtfilt(*butterworth_filter, df["x_p3"])


### Gaussian filter

In [ ]:
from scipy.ndimage import gaussian_filter1d

df["x_p1_filt_gaussian"] = gaussian_filter1d(df["x_p1"], 1) 
df["x_p2_filt_gaussian"] = gaussian_filter1d(df["x_p2"], 1)
df["x_p3_filt_gaussian"] = gaussian_filter1d(df["x_p3"], 1)


In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scattergl(
    x=df['n'],
    y=df['x_p1'],
    mode='markers+lines',
    name='Prox 1 unfiltered'
), secondary_y=False)
fig.add_trace(go.Scattergl(
    x=df['n'],
    y=df['x_p1_filt_butter'],
    mode='markers+lines',
    name='Prox 1 filtered (butterworth)'
), secondary_y=False)
fig.add_trace(go.Scattergl(
    x=df['n'],
    y=df['x_p1_filt_gaussian'],
    mode='markers+lines',
    name='Prox 1 filtered (gaussian)'
), secondary_y=False)

fig.add_trace(go.Scattergl(
    x=df['n'],
    y=df['Omega']*60/(2*np.pi),
    mode='markers+lines',
    name='Shaft speed'
), secondary_y=True)

fig.update_layout(
    title='Filtered tip deflections',
    xaxis_title='Shaft revolution',
    yaxis_title='x [μm]',
    legend_title='Filter type'
)
fig.show()


## Peak to Peak vibration

In [ ]:
x_matrix = (
    df[["x_p1_filt_butter", "x_p2_filt_butter", "x_p3_filt_butter"]]
    .to_numpy()
)
df["pk-pk"] = x_matrix.max(axis=1) - x_matrix.min(axis=1)


In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scattergl(
    x=df['n'],
    y=df['pk-pk'],
    mode='markers+lines',
    name='Prox 1 unfiltered'
), secondary_y=False)

fig.add_trace(go.Scattergl(
    x=df['n'],
    y=df['Omega']*60/(2*np.pi),
    mode='markers+lines',
    name='Shaft speed'
), secondary_y=True)

fig.update_layout(
    title='Peak-to-peak tip deflections',
    xaxis_title='Shaft revolution',
    yaxis_title='pk-pk [μm]'
)
fig.show()


### Peak to peak vibration vs shaft speed

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scattergl(
    x=df['Omega']*60/(2*np.pi),
    y=df['pk-pk'],
    mode='markers',
    name='Peak to Peak vibration'
))

fig.update_layout(
    title='Peak-to-peak tip deflections',
    xaxis_title='Shaft speed',
    yaxis_title='pk-pk [μm]'
)
fig.show()


